In [3]:
import os
import numpy as np
from shutil import copyfile

Path = "/home/ug-ml/felix-ML/TrainingDataRaw"

NewPath = "/home/ug-ml/felix-ML/DataGenerator3/Data"
CifFlolder = "/home/ug-ml/felix-ML/DataGenerator3/CifFolder"

def ThicknessString(String): #Takes the file name string, looks at the thickness part and returns a integer corresponding to the thickness class
    if(String == "0050"):
        return 0
    elif(String == "0250"):
        return 1
    elif(String == "0450"):
        return 2
    elif(String == "0650"):
        return 3
    elif(String == "0850"):
        return 4
    elif(String == "1050"):
        return 5
    elif(String == "1250"):
        return 6
    elif(String == "1450"):
        return 7
    elif(String == "1650"):
        return 8
    elif(String == "1850"):
        return 9
    else: #If something goes wrong, an error will be printed
        print("ERROR")
        return 10

def Position():
    IndexArray = np.zeros(8 * 8, dtype = int).reshape(8, 8) #Declare Image array
    index = 0
    for i in range(0, 8):
        for j in range(0, 8):
            if(i >= j):
                IndexArray[i][j] = index
                index+=1
    return IndexArray

DirectionIndex = Position()




def WorkingCrystalIndex(Path):
    WorkingCrystalIndex = []
    Path_i = sorted(os.listdir(Path))
    for i in Path_i:
        Thousand = int(i.split("to")[0]) - 1
        Path_j = sorted((os.listdir(Path + "/" + i)))
        for j in Path_j:
            Path_k = sorted((os.listdir(Path + "/" + i + "/" + j)))
            if(len(Path_k) != 13):
                print("These files did not work beacause there is no information (length, name) = ", len(Path_k), j)
            else:
                WorkingCrystalIndex.append(int(j))
    return(WorkingCrystalIndex)
                

def ShuffleIndexCreator(WorkingCrystalIndex):
    rng = np.random.default_rng()
    length = len(WorkingCrystalIndex)
    ShuffleIndex = np.arange(length, dtype = np.int)
    rng.shuffle(ShuffleIndex)
    return(ShuffleIndex)
    
            
            
            
def BinToDirectory(Path, NewPath, CifFlolder, ImageArray, ShuffleIndex, TrainingRatio, File1, DirectionIndex):
    Thousand = 0
    Iteration = 0
    NumberOfCrystals = len(ShuffleIndex)
    NumberTraining = int(NumberOfCrystals * TrainingRatio)
    InTraining = True
    Path_i = sorted(os.listdir(Path))
    for i in Path_i:
        Thousand = int(i.split("to")[0]) - 1
        Path_j = sorted((os.listdir(Path + "/" + i)))
        for j in Path_j:
            print(int(j))
            Path_k = sorted((os.listdir(Path + "/" + i + "/" + j)))
            if(len(Path_k) != 13):
                print("These files did not work beacause there is no information (length, name) = ", len(Path_k), j)
            else:
                for k in Path_k:
                    if(k[-4:] == ".cif"):
                        #CRYSTAL FILE INFOMATION
                        CifOpener(Path + "/" + i + "/" + j + "/" + k, CifFlolder, int(j), File1)
                    elif(k[-7:] == "128x128"):
                        #Folders containing image files
                        RandomIndex = ShuffleIndex[Iteration]
                        if(RandomIndex < NumberTraining):
                            InTraining = True
                        else:
                            InTraining = False
                        BinToNpy(Path + "/" + i + "/" + j + "/" + k, NewPath, int(j), ImageArray, InTraining, DirectionIndex)
                    elif(k[-4:] == ".hkl" or k[-4:] == ".inp"):
                        n = 3
                        #Do nothing with currently
                    else:
                        print("File or folder with name (Warning1):", k)
                        
                
                Iteration+=1
                    

def CifOpener(Path, CifFlolder, CrystalNo, File1):
    lattice_a = 0
    with open(Path) as textFile:
        lines = [line.split() for line in textFile]

    for i in lines:
        for j in range(0, len(i)):
            if(i[j] == "_cell_length_a"):
                lattice_a = float(i[j + 1].split("(")[0])
                #print(lattice_a)
                break
    
    File1.write(str(CrystalNo) + " " + str(lattice_a) + "\n")
    copyfile(Path,CifFlolder + "/" + str(CrystalNo) + ".cif")
    
    
    
def BinToNpy(BinFilePath, NewPath, CrystalNo, ImageArray, InTraining, DirectionIndex):
    DataSets = ["/training", "/validation"]
    Classification = ["/0", "/1", "/2", "/3", "/4", "/5", "/6", "/7", "/8", "/9"]
    CrystalName = "/Crystal_"
    #Open thickness folder
    NumberOfCrystals = len(ShuffleIndex)
    ThicknessName = BinFilePath.split("_")[-2][0:-1]
    Thickness_i = ThicknessString(ThicknessName)
    #print("Thickness name", Thickness_i)
    Path_l = sorted((os.listdir(BinFilePath)))
    BinFiles = []
    for l in Path_l:
        if(l[-4:] == ".bin"):
            #Load bin file
            BinFiles.append(l)
          
        elif(l[-4:] == ".cif" or l[-4:] == ".gif" or l[-4:] == ".txt"):
            #Ignore
            m = 2
        else:
            print("File or folder with name (Warning2):", l)
    #print(BinFiles)
    
    for i in BinFiles:
        DirectionName = list(map(int, i.split("_")[-1].split(".")[0].split("+")[1:]))
        #print(DirectionName)
        if(DirectionName[0] <= 14 and DirectionName[1] <=14):
            Directon_i = DirectionIndex[int(DirectionName[0]/2)][int(DirectionName[1]/2)]
            print(Directon_i)
            data = np.fromfile(BinFilePath + "/" + i, dtype=np.float64).reshape(128, 128)
            ImageArray[Directon_i] = data.astype(np.float32)
    
    
    if(InTraining == True):
        np.save(NewPath + DataSets[0] + Classification[Thickness_i] + CrystalName + str(CrystalNo) + ".npy", ImageArray)
    else:
        np.save(NewPath + DataSets[1] + Classification[Thickness_i] + CrystalName + str(CrystalNo) + ".npy", ImageArray)
    



In [4]:
TrainingRatio = 0.9

WorkingCrystalIndex = WorkingCrystalIndex(Path)

ShuffleIndex = ShuffleIndexCreator(WorkingCrystalIndex)

ImageArray = np.zeros(36 * 128 * 128, dtype = np.float32).reshape(36, 128, 128)

File1  = open(CifFlolder +"/CifKey1_" + ".txt", "a")

BinToDirectory(Path, NewPath, CifFlolder, ImageArray, ShuffleIndex, TrainingRatio, File1, DirectionIndex)

File1.close()

These files did not work beacause there is no information (length, name) =  3 1122
These files did not work beacause there is no information (length, name) =  3 1442
These files did not work beacause there is no information (length, name) =  3 1577
These files did not work beacause there is no information (length, name) =  3 1792
These files did not work beacause there is no information (length, name) =  3 1829
These files did not work beacause there is no information (length, name) =  3 1831
These files did not work beacause there is no information (length, name) =  3 1865
These files did not work beacause there is no information (length, name) =  3 1866
These files did not work beacause there is no information (length, name) =  3 2111
These files did not work beacause there is no information (length, name) =  3 2343
These files did not work beacause there is no information (length, name) =  3 2998
1001
0
15
20
16
17
18
19
21
26
27
22
23
24
25
28
33
34
35
29
30
31
32
1
2
3
4
5
6
7
8
9

30
31
32
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
15
20
16
17
18
19
21
26
27
22
23
24
25
28
33
34
35
29
30
31
32
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
15
20
16
17
18
19
21
26
27
22
23
24
25
28
33
34
35
29
30
31
32
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
15
20
16
17
18
19
21
26
27
22
23
24
25
28
33
34
35
29
30
31
32
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1009
0
15
20
16
17
18
19
21
26
27
22
23
24
25
28
33
34
35
29
30
31
32
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
15
20
16
17
18
19
21
26
27
22
23
24
25
28
33
34
35
29
30
31
32
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
15
20
16
17
18
19
21
26
27
22
23
24
25
28
33
34
35
29
30
31
32
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
15
20
16
17
18
19
21
26
27
22
23
24
25
28
33
34
35
29
30
31
32
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
15
20
16
17
18
19
21
26
27
22
23
24
25
28
33
34
35
29
30
31
32
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
15
20
16
17
18
19
21
26
27
22
23
24
25
28
33
34
35
29
30
31
32
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
15
20
16
17
18
19
21
26
27
22
23
24
25
28
33
34
35
29
30
31
32
1
2
3


KeyboardInterrupt: 